<a href="https://colab.research.google.com/github/yy0750/KOSPI200-Components-Direction-Prediction/blob/main/predict_one_month.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.8 MB/s eta 0:00:00


In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/code/통계학과머신러닝PBL
!pwd

import pandas as pd
import numpy as np
import random
import time
import pickle
from sklearn.preprocessing import OneHotEncoder
from Statistics import Statistics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb

import os
# 시드 값 설정
SEED = 9
# 파이썬 해시 함수의 시드를 설정하여 해시 값이 재현 가능하도록 합니다.
os.environ['PYTHONHASHSEED'] = str(SEED)
# 파이썬의 내장 random 모듈에서 사용하는 시드 값을 설정합니다.
random.seed(SEED)
# NumPy의 random 모듈에서 사용하는 시드 값을 설정합니다.
np.random.seed(SEED)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/code/통계학과머신러닝PBL'
/content/drive/MyDrive/code/통계학과머신러닝PBL
/content/drive/My Drive/code/통계학과머신러닝PBL


In [ ]:
# 일일 수익률 계산, 중앙값 기준 이진 라벨링
def create_label(df):
    perc = [0.0, 0.5, 1.0]
    label = df.iloc[:, 1:].pct_change(periods=20, fill_method=None)[20:].apply(
        lambda x: pd.qcut(x.rank(method='first'), perc, labels=False), axis=1)
    return label

# 주식 데이터 생성
def create_stock_data(df, st, test_year):
    st_data = pd.DataFrame([])
    st_data['Date'] = list(df['Date'])
    st_data['Name'] = [st] * len(st_data)

    for k in list(range(1, 21)) + list(range(40, 241, 20)):
        st_data['R' + str(k)] = df[st].pct_change(k)

    st_data['R-future'] = df[st].pct_change(20).shift(-20)
    st_data['label'] = list(label[st])+ [np.nan]*20
    st_data['Date'] = st_data['Date'].astype(str)
    st_data['Month'] = list(df['Date'].str[:-3])
    st_data = st_data.dropna()

    trade_year = st_data['Month'].str[:4]
    st_data = st_data.drop(columns=['Month'])

    st_train_data = st_data[trade_year < str(test_year)]
    st_test_data = st_data[trade_year == str(test_year)]

    return np.array(st_train_data), np.array(st_test_data), st_data


def trainer(train_data, test_data):
    random.seed(SEED)
    np.random.seed(SEED)

    train_x, train_y = train_data[:, 2:-2], train_data[:, -1]
    train_y = train_y.astype('int')

    print('Started training')
    #clf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=SEED, n_jobs=-1)
    #clf = LogisticRegression(random_state=SEED, max_iter=1000, n_jobs=-1)
    #clf = XGBClassifier(n_estimators=1000, max_depth=20, random_state=SEED, n_jobs=-1,)
    clf = CatBoostClassifier(iterations=1000, depth=16, random_seed=SEED, thread_count=-1, verbose=False)
    #clf = lgb.LGBMClassifier(n_estimators=1000, max_depth=20, random_state=SEED, n_jobs=-1)
    clf.fit(train_x, train_y)
    print('Completed ', clf.score(train_x, train_y))

    dates = list(set(test_data[:, 0]))
    predictions = {}

    for day in dates:
        test_d = test_data[test_data[:, 0] == day]
        test_d = test_d[:, 2:-2]
        predictions[day] = clf.predict_proba(test_d)[:, 1]

    return predictions

# def simulate(test_data,predictions):
#     rets = pd.DataFrame([],columns=['Long'])
#     k = 2
#     for day in sorted(predictions.keys()):
#         preds = predictions[day]
#         test_returns = test_data[test_data[:,0]==day][:,-2]
#         top_preds = predictions[day].argsort()[-k:][::-1]
#         trans_long = test_returns[top_preds]

#         recommended_stock = test_data[test_data[:, 0] == day][top_preds, 1][0]
#         print(recommended_stock)
#         rets.loc[day] = [np.mean(trans_long)]
#     return rets

def simulate(test_data, predictions):
    rets = pd.DataFrame(columns=['Date', 'Long', 'Stock1', 'Stock2'])
    k = 2
    for day in sorted(predictions.keys()):
        preds = predictions[day]
        test_returns = test_data[test_data[:, 0] == day][:, -2]
        top_preds = predictions[day].argsort()[-k:][::-1]
        trans_long = test_returns[top_preds]

        # 추천된 두 종목의 이름을 가져오기
        recommended_stocks = test_data[test_data[:, 0] == day][top_preds, 1]

        new_row = pd.DataFrame({
            'Date': [day],
            'Long': [np.mean(trans_long)],
            'Stock1': [recommended_stocks[0]],
            'Stock2': [recommended_stocks[1]]
        })
        rets = pd.concat([rets, new_row], ignore_index=True)
    return rets

result_folder = 'test'
for directory in [result_folder]:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
test_year = 2024

print('-' * 40)
print("test_year: ", test_year)
print('-' * 40)

# 파일 경로 설정
df = pd.read_csv('/content/drive/MyDrive/code/통계학과머신러닝PBL/data/Close2.csv')
KS200_df = pd.read_csv('/content/drive/MyDrive/code/통계학과머신러닝PBL/data/KSXconst.csv', dtype=str)
all_companies =  df.columns[1:].tolist()

# 주어진 주식 리스트
stock_list = [
    '001570', '009420', '000990', '034020', '128940',
    '052690', '003230', '006650', '017800', '069620'
]

# 키를 '2023-12'로 설정하여 딕셔너리 생성
constituents = {'2023-12': set(stock_list)}

# 결과 출력
print(constituents)

label = create_label(df)
train_data, test_data = [], []  # 빈 리스트로 초기화


for st in stock_list:
    st_train_data, st_test_data, st_data = create_stock_data(df, st, test_year)
    train_data.append(st_train_data)
    test_data.append(st_test_data)

train_data = np.concatenate([x for x in train_data])
test_data = np.concatenate([x for x in test_data])
print('Created :', train_data.shape, test_data.shape)

predictions = trainer(train_data,test_data)
returns = simulate(test_data, predictions)

# result = Statistics(returns.sum(axis=1))
# print('\nAverage returns prior to transaction charges')

returns.to_csv(result_folder + '/avg_daily_rets-' + str(test_year) + '.csv')
returns

----------------------------------------
test_year:  2024
----------------------------------------
{'2023-12': {'017800', '069620', '009420', '052690', '128940', '000990', '001570', '003230', '006650', '034020'}}
Created : (29620, 35) (860, 35)
Started training


In [ ]:
test_data

array([['2024-01-02', '001570', -0.030219780219780223, ...,
        3.2023809523809526, -0.26345609065155806, 0.0],
       ['2024-01-03', '001570', -0.017941454202077378, ...,
        3.193548387096774, -0.24903846153846154, 0.0],
       ['2024-01-04', '001570', -0.011538461538461497, ...,
        3.055226824457594, -0.2324902723735408, 0.0],
       ...,
       ['2024-05-03', '069620', 0.001790510295434089, ...,
        0.0026881720430107503, -0.025915996425379784, 0.0],
       ['2024-05-07', '069620', 0.0044682752457552155, ...,
        -0.000888888888888939, -0.03380782918149461, 0.0],
       ['2024-05-08', '069620', 0.002669039145907437, ...,
        0.001777777777777878, -0.032830523513753374, 0.0]], dtype=object)